# 39C3 APIv2 Endpoint Testing

This notebook tests all endpoints of the 39C3 APIv2 (except ping).

## API Overview
The 39C3 APIv2 provides access to:
- Conference information and tracks
- Assemblies and their rooms
- Events (official, assembly, sos, lightning)
- Projects
- Users
- Badges

## Base URL
The API is available at: `https://api.events.ccc.de/congress/2025/v2/`

## Authentication
This notebook loads the API key from environment variables. Set one of the following in your `.env` file:
- `39C3_API_KEY`
- `C3_API_KEY`
- `EVENTS_CCC_API_KEY`
- `SECRET__39C3__API_KEY`
- `SECRET__C3__API_KEY`

The API key will be sent as a Bearer token in the Authorization header.

In [1]:
import requests
import json
import os
import pathlib
from typing import Optional, Dict, Any
from datetime import datetime, timedelta
from dotenv import load_dotenv

# ---------- Load .env (search upwards) ----------
# Load environment variables from .env file (similar to youtubetranscript.ipynb)
cwd = pathlib.Path.cwd()
env_path = None
for parent in [cwd] + list(cwd.parents):
    candidate = parent / ".env"
    if candidate.is_file():
        env_path = candidate
        break

if env_path is None:
    print("⚠️  No .env file found – you must set env vars manually.")
else:
    print(f"🔑 Loading environment from {env_path}")
    load_dotenv(env_path)

# ---------- Get API Key from environment ----------
# Try common environment variable names for 39C3 API key
API_KEY = (
    os.getenv("39C3_API_KEY") or
    os.getenv("C3_API_KEY") or
    os.getenv("EVENTS_CCC_API_KEY") or
    os.getenv("SECRET__39C3__API_KEY") or
    os.getenv("SECRET__C3__API_KEY")
)

if not API_KEY:
    print("⚠️  No API key found in environment variables.")
    print("   Please set one of: 39C3_API_KEY, C3_API_KEY, EVENTS_CCC_API_KEY, SECRET__39C3__API_KEY, or SECRET__C3__API_KEY")
else:
    print(f"✅ API key loaded (length: {len(API_KEY)} characters)")

# Base URL for the 39C3 APIv2
# Note: The API is at api.events.ccc.de, not events.ccc.de
BASE_URL = "https://api.events.ccc.de/congress/2025/v2"

def make_request(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    """
    Make a GET request to the API endpoint.
    
    Args:
        endpoint: The API endpoint path (without base URL)
        params: Optional query parameters
        
    Returns:
        Response JSON as dictionary, or None if request failed
    """
    url = f"{BASE_URL}{endpoint}"
    
    # Prepare headers with authentication
    headers = {}
    if API_KEY:
        # Most APIs use Bearer token authentication
        headers["Authorization"] = f"Bearer {API_KEY}"
    
    try:
        response = requests.get(url, params=params, headers=headers)
        
        # Debug: Print response details
        print(f"   Status Code: {response.status_code}")
        print(f"   Content-Type: {response.headers.get('Content-Type', 'N/A')}")
        print(f"   Response Length: {len(response.content)} bytes")
        
        # Check if response is actually JSON
        content_type = response.headers.get('Content-Type', '').lower()
        if 'application/json' not in content_type:
            print(f"   ⚠️  Response is not JSON! Content-Type: {content_type}")
            print(f"   First 500 chars of response: {response.text[:500]}")
            return None
        
        response.raise_for_status()
        
        # Try to parse JSON
        try:
            return response.json()
        except json.JSONDecodeError as json_err:
            print(f"   ❌ JSON decode error: {json_err}")
            print(f"   Response text (first 500 chars): {response.text[:500]}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Error requesting {url}: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"   Status Code: {e.response.status_code}")
            print(f"   Response: {e.response.text[:500]}")
        return None

def print_response(title: str, data: Optional[Dict[str, Any]]):
    """
    Pretty print API response.
    
    Args:
        title: Section title
        data: Response data dictionary
    """
    print(f"\n{'='*60}")
    print(f"📋 {title}")
    print(f"{'='*60}")
    
    if data is None:
        print("❌ No data returned")
        return
    
    # Pretty print JSON
    print(json.dumps(data, indent=2, ensure_ascii=False))
    
    # If it's a list response, show summary
    if isinstance(data, dict) and 'data' in data and isinstance(data['data'], list):
        items = data['data']
        print(f"\n📊 Summary: {len(items)} items returned")
        if 'pagination' in data:
            pagination = data['pagination']
            total_pages = (pagination.get('total', 0) + pagination.get('page_size', 25) - 1) // pagination.get('page_size', 25)
            print(f"   Page: {pagination.get('page', 'N/A')}/{total_pages if total_pages > 0 else 1}")
            print(f"   Total: {pagination.get('total', 0)} items")
    
    print()

# Test results storage
test_results = {}

# First, let's test the ping endpoint to verify the API is accessible
print("🔍 Testing API connectivity with ping endpoint...")
try:
    ping_response = requests.get(f"{BASE_URL}/ping/", timeout=10)
    print(f"   Status Code: {ping_response.status_code}")
    print(f"   Content-Type: {ping_response.headers.get('Content-Type', 'N/A')}")
    print(f"   Response: {ping_response.text[:200]}")
    if ping_response.status_code == 200:
        print("   ✅ Ping successful!")
    else:
        print(f"   ⚠️  Unexpected status code")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n🔍 Testing alternative base URLs...")
# Try different base URLs to find the correct one
alt_urls = [
    "https://api.events.ccc.de/congress/2025/v2",  # Correct API URL
    "https://events.ccc.de/congress/2025/v2",      # Wrong (returns HTML)
    "https://events.ccc.de/v2",
]

for alt_url in alt_urls:
    try:
        test_response = requests.get(f"{alt_url}/ping/", timeout=5)
        print(f"   {alt_url}/ping/ -> Status: {test_response.status_code}, Content-Type: {test_response.headers.get('Content-Type', 'N/A')}")
        if test_response.status_code == 200:
            content_type = test_response.headers.get('Content-Type', '').lower()
            if 'application/json' in content_type or test_response.text.strip() == '"pong"':
                print(f"      ✅ This URL works! Response: {test_response.text[:100]}")
            else:
                print(f"      ⚠️  Returns HTML, not JSON")
    except Exception as e:
        print(f"   {alt_url}/ping/ -> Error: {str(e)[:50]}")
print()

🔑 Loading environment from /home/superdev/projects/OpenMates/.env
✅ API key loaded (length: 56 characters)
🔍 Testing API connectivity with ping endpoint...
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response: "pong"
   ✅ Ping successful!

🔍 Testing alternative base URLs...
   https://api.events.ccc.de/congress/2025/v2/ping/ -> Status: 200, Content-Type: application/json; charset=utf-8
      ✅ This URL works! Response: "pong"
   https://events.ccc.de/congress/2025/v2/ping/ -> Status: 200, Content-Type: text/html; charset=utf-8
      ⚠️  Returns HTML, not JSON
   https://events.ccc.de/v2/ping/ -> Status: 404, Content-Type: text/html; charset=utf-8



## 1. Conference Endpoints

In [2]:
# 1.1 Get Conference Information
print("🔍 Testing: GET /conference/")
conference_data = make_request("/conference/")
test_results['conference'] = conference_data
print_response("Conference Information", conference_data)

# Extract conference ID if available for reference
conference_id = None
if conference_data and 'data' in conference_data:
    conference_id = conference_data['data'].get('id')
    if conference_id:
        print(f"✅ Conference ID: {conference_id}")

🔍 Testing: GET /conference/
   Status Code: 500
   Content-Type: text/html; charset=utf-8
   Response Length: 145 bytes
   ⚠️  Response is not JSON! Content-Type: text/html; charset=utf-8
   First 500 chars of response: 
<!doctype html>
<html lang="en">
<head>
  <title>Server Error (500)</title>
</head>
<body>
  <h1>Server Error (500)</h1><p></p>
</body>
</html>


📋 Conference Information
❌ No data returned


In [3]:
# 1.2 List Conference Tracks
print("🔍 Testing: GET /conference/tracks/")
tracks_data = make_request("/conference/tracks/", params={"pageSize": 10})
test_results['tracks_list'] = tracks_data
print_response("Conference Tracks List", tracks_data)

# Extract first track ID for detail endpoint
first_track_id = None
if tracks_data and 'data' in tracks_data and len(tracks_data['data']) > 0:
    first_track_id = tracks_data['data'][0].get('id')
    print(f"✅ First Track ID: {first_track_id}")

🔍 Testing: GET /conference/tracks/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 776 bytes

📋 Conference Tracks List
{
  "data": [
    {
      "banner_image": null,
      "slug": "security",
      "name": "Security",
      "color": "#0347b4",
      "id": 1
    },
    {
      "banner_image": null,
      "slug": "hardware",
      "name": "Hardware",
      "color": "#685b9d",
      "id": 2
    },
    {
      "banner_image": null,
      "slug": "ccc-community",
      "name": "CCC & Community",
      "color": "#a4a300",
      "id": 3
    },
    {
      "banner_image": null,
      "slug": "entertainment",
      "name": "Entertainment",
      "color": "#4D4D4C",
      "id": 4
    },
    {
      "banner_image": null,
      "slug": "art-beauty",
      "name": "Art & Beauty",
      "color": "#f9b000",
      "id": 5
    },
    {
      "banner_image": null,
      "slug": "ethics-society-politics",
      "name": "Ethics, Society & Politics",
      "color":

In [4]:
# 1.3 Get Conference Track by ID
if first_track_id:
    print(f"🔍 Testing: GET /conference/tracks/{first_track_id}/")
    track_detail = make_request(f"/conference/tracks/{first_track_id}/")
    test_results['track_detail'] = track_detail
    print_response(f"Track Detail (ID: {first_track_id})", track_detail)
else:
    print("⚠️  Skipping track detail test - no track ID available")

🔍 Testing: GET /conference/tracks/1/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 101 bytes

📋 Track Detail (ID: 1)
{
  "data": {
    "banner_image": null,
    "slug": "security",
    "name": "Security",
    "color": "#0347b4",
    "id": 1
  }
}



## 2. Assembly Endpoints

In [5]:
# 2.1 List Assemblies
print("🔍 Testing: GET /assemblies/")
assemblies_data = make_request("/assemblies/", params={
    "pageSize": 10,
    "state": ["accepted", "placed", "arrived", "confirmed"]  # Public states only
})
test_results['assemblies_list'] = assemblies_data
print_response("Assemblies List", assemblies_data)

# Extract first assembly ID for detail and rooms endpoints
first_assembly_id = None
if assemblies_data and 'data' in assemblies_data and len(assemblies_data['data']) > 0:
    first_assembly_id = assemblies_data['data'][0].get('id')
    print(f"✅ First Assembly ID: {first_assembly_id}")

🔍 Testing: GET /assemblies/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 19535 bytes

📋 Assemblies List
{
  "data": [
    {
      "tags": [
        "luebeck",
        "nbsp",
        "nobreakspace",
        "lubeck"
      ],
      "description": {
        "preferred": "Chaotikum e.V. is a non-profit association founded in March 2012 with the primary goal of providing a space for tech enthusiasts in and around Lübeck to tinker and chat.\r\nThis space, called the Nobreakspace, is located at Fackenburger Allee 11 (23554, Lübeck). Here, we meet to work on various projects, discuss topics that interest us, and, most importantly, have fun. You can check if the Nobreakspace is currently open on our status page. Everyone is welcome to stop by!",
        "de": "Chaotikum e.V. ist ein im März 2012 gegrüdeter gemeinnütziger Verein mit dem Hauptziel, technikinteressierten Menschen in und um Lübeck einen Ort zum Basteln und Quatschen zu geben.\r\nDieser O

In [6]:
# 2.2 Get Assembly by ID
if first_assembly_id:
    print(f"🔍 Testing: GET /assemblies/{first_assembly_id}/")
    assembly_detail = make_request(f"/assemblies/{first_assembly_id}/")
    test_results['assembly_detail'] = assembly_detail
    print_response(f"Assembly Detail (ID: {first_assembly_id})", assembly_detail)
else:
    print("⚠️  Skipping assembly detail test - no assembly ID available")

🔍 Testing: GET /assemblies/000e8037-1af2-4951-8d4d-58737b6d8ad5/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 2043 bytes

📋 Assembly Detail (ID: 000e8037-1af2-4951-8d4d-58737b6d8ad5)
{
  "data": {
    "tags": [
      "luebeck",
      "nbsp",
      "nobreakspace",
      "lubeck"
    ],
    "description": {
      "preferred": "Chaotikum e.V. is a non-profit association founded in March 2012 with the primary goal of providing a space for tech enthusiasts in and around Lübeck to tinker and chat.\r\nThis space, called the Nobreakspace, is located at Fackenburger Allee 11 (23554, Lübeck). Here, we meet to work on various projects, discuss topics that interest us, and, most importantly, have fun. You can check if the Nobreakspace is currently open on our status page. Everyone is welcome to stop by!",
      "de": "Chaotikum e.V. ist ein im März 2012 gegrüdeter gemeinnütziger Verein mit dem Hauptziel, technikinteressierten Menschen in und um Lübeck ei

In [7]:
# 2.3 List Assembly Rooms
if first_assembly_id:
    print(f"🔍 Testing: GET /assemblies/{first_assembly_id}/rooms/")
    rooms_data = make_request(f"/assemblies/{first_assembly_id}/rooms/", params={"pageSize": 10})
    test_results['assembly_rooms_list'] = rooms_data
    print_response(f"Assembly Rooms List (Assembly ID: {first_assembly_id})", rooms_data)
    
    # Extract first room ID for detail endpoint
    first_room_id = None
    if rooms_data and 'data' in rooms_data and len(rooms_data['data']) > 0:
        first_room_id = rooms_data['data'][0].get('id')
        print(f"✅ First Room ID: {first_room_id}")
else:
    print("⚠️  Skipping assembly rooms list test - no assembly ID available")
    first_room_id = None

🔍 Testing: GET /assemblies/000e8037-1af2-4951-8d4d-58737b6d8ad5/rooms/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 68 bytes

📋 Assembly Rooms List (Assembly ID: 000e8037-1af2-4951-8d4d-58737b6d8ad5)
{
  "data": [],
  "pagination": {
    "page_size": 10,
    "page": 1,
    "total": 0
  }
}

📊 Summary: 0 items returned
   Page: 1/1
   Total: 0 items



In [8]:
# 2.4 Get Assembly Room by ID
if first_assembly_id and first_room_id:
    print(f"🔍 Testing: GET /assemblies/{first_assembly_id}/rooms/{first_room_id}/")
    room_detail = make_request(f"/assemblies/{first_assembly_id}/rooms/{first_room_id}/")
    test_results['assembly_room_detail'] = room_detail
    print_response(f"Assembly Room Detail (Assembly ID: {first_assembly_id}, Room ID: {first_room_id})", room_detail)
else:
    print("⚠️  Skipping assembly room detail test - missing assembly or room ID")

⚠️  Skipping assembly room detail test - missing assembly or room ID


## 3. Event Endpoints

In [9]:
# 3.1 List Events
print("🔍 Testing: GET /events/")
events_data = make_request("/events/", params={
    "pageSize": 10,
    "kind": "official"  # Test filtering by kind
})
test_results['events_list'] = events_data
print_response("Events List", events_data)

# Extract first event ID for detail endpoint
first_event_id = None
if events_data and 'data' in events_data and len(events_data['data']) > 0:
    first_event_id = events_data['data'][0].get('id')
    print(f"✅ First Event ID: {first_event_id}")

🔍 Testing: GET /events/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 75685 bytes

📋 Events List
{
  "data": [
    {
      "track_id": 1,
      "assembly_id": "2465345f-f0e4-4a72-96d8-346d703a59e3",
      "room_id": "7202df07-050c-552f-8318-992f94e40ef0",
      "description": {
        "preferred": "The Freebox HD is a set-top box with media player capabilities designed and built by the French ISP 'Free' in 2006, and distributed to customers since (including me). It is still in use and will be maintained until the end of 2025.\r\n\r\nWhen I got it, I wanted to run homebrew software on it, so I decided to reverse engineer it. The initial goal was to get arbitrary code execution. The Freebox HD being largely undocumented, this talk shows the full process of reverse engineering it from scratch:\r\n* Initial visual inspection\r\n* Disassembly and inspection of the insides\r\n* Attack surface analysis and choice of the target\r\n* Search and exploi

In [10]:
# 3.2 Get Event by ID
if first_event_id:
    print(f"🔍 Testing: GET /events/{first_event_id}/")
    event_detail = make_request(f"/events/{first_event_id}/")
    test_results['event_detail'] = event_detail
    print_response(f"Event Detail (ID: {first_event_id})", event_detail)
else:
    print("⚠️  Skipping event detail test - no event ID available")

🔍 Testing: GET /events/032fdd30-9488-55b8-968c-dbce19a3f446/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 5820 bytes

📋 Event Detail (ID: 032fdd30-9488-55b8-968c-dbce19a3f446)
{
  "data": {
    "track_id": 1,
    "assembly_id": "2465345f-f0e4-4a72-96d8-346d703a59e3",
    "room_id": "7202df07-050c-552f-8318-992f94e40ef0",
    "description": {
      "preferred": "The Freebox HD is a set-top box with media player capabilities designed and built by the French ISP 'Free' in 2006, and distributed to customers since (including me). It is still in use and will be maintained until the end of 2025.\r\n\r\nWhen I got it, I wanted to run homebrew software on it, so I decided to reverse engineer it. The initial goal was to get arbitrary code execution. The Freebox HD being largely undocumented, this talk shows the full process of reverse engineering it from scratch:\r\n* Initial visual inspection\r\n* Disassembly and inspection of the insides\r\n* Attack 

In [11]:
# 3.3 Test Event Filtering Examples
print("🔍 Testing: Event filtering examples")

# Filter by assembly
if first_assembly_id:
    print(f"\n📌 Filtering events by assembly ID: {first_assembly_id}")
    events_by_assembly = make_request("/events/", params={
        "assemblyID": [first_assembly_id],
        "pageSize": 5
    })
    if events_by_assembly and 'data' in events_by_assembly:
        print(f"   Found {len(events_by_assembly['data'])} events for this assembly")

# Filter by date range (example: events starting after a specific date)
print(f"\n📌 Filtering events by date range")
start_date = (datetime.now() - timedelta(days=30)).isoformat()
events_by_date = make_request("/events/", params={
    "startAfter": start_date,
    "pageSize": 5
})
if events_by_date and 'data' in events_by_date:
    print(f"   Found {len(events_by_date['data'])} events starting after {start_date}")

# Filter by language
print(f"\n📌 Filtering events by language: 'de'")
events_by_lang = make_request("/events/", params={
    "language": ["de"],
    "pageSize": 5
})
if events_by_lang and 'data' in events_by_lang:
    print(f"   Found {len(events_by_lang['data'])} events in German")

🔍 Testing: Event filtering examples

📌 Filtering events by assembly ID: 000e8037-1af2-4951-8d4d-58737b6d8ad5
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 67 bytes
   Found 0 events for this assembly

📌 Filtering events by date range
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 45959 bytes
   Found 5 events starting after 2025-11-07T15:20:57.744192

📌 Filtering events by language: 'de'
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 34444 bytes
   Found 5 events in German


## 4. Project Endpoints

In [12]:
# 4.1 List Projects
print("🔍 Testing: GET /projects/")
projects_data = make_request("/projects/", params={"pageSize": 10})
test_results['projects_list'] = projects_data
print_response("Projects List", projects_data)

# Extract first project ID for detail endpoint
first_project_id = None
if projects_data and 'data' in projects_data and len(projects_data['data']) > 0:
    first_project_id = projects_data['data'][0].get('id')
    print(f"✅ First Project ID: {first_project_id}")

🔍 Testing: GET /projects/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 18057 bytes

📋 Projects List
{
  "data": [
    {
      "assembly_id": "0233ea36-ba76-496e-80a6-a67dd6b50b6d",
      "description": {
        "preferred": "The Hakkaa board is the perfect starter project for SMD soldering, even if you've never soldered before. Small and surprisingly simple. Once you've soldered your first components, you'll quickly realise that SMD soldering is not rocket science.\r\n\r\nThe board brings Persistence of Vision (POV) to life: fast LED sequences create floating text and patterns in the air. Everything is controlled by a RISC-V microcontroller whose firmware is written in Embedded Rust. Rust makes programming super intuitive. Anyone who has ever written Arduino code will love it. Text, symbols and animations can be easily customised without having to struggle through complicated toolchains.\r\n\r\nLEDs, microcontrollers, Rust code... all on one

In [13]:
# 4.2 Get Project by ID
if first_project_id:
    print(f"🔍 Testing: GET /projects/{first_project_id}/")
    project_detail = make_request(f"/projects/{first_project_id}/")
    test_results['project_detail'] = project_detail
    print_response(f"Project Detail (ID: {first_project_id})", project_detail)
else:
    print("⚠️  Skipping project detail test - no project ID available")

🔍 Testing: GET /projects/05667d15-de3a-4a83-88cb-5c88aa922728/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 3211 bytes

📋 Project Detail (ID: 05667d15-de3a-4a83-88cb-5c88aa922728)
{
  "data": {
    "assembly_id": "0233ea36-ba76-496e-80a6-a67dd6b50b6d",
    "description": {
      "preferred": "The Hakkaa board is the perfect starter project for SMD soldering, even if you've never soldered before. Small and surprisingly simple. Once you've soldered your first components, you'll quickly realise that SMD soldering is not rocket science.\r\n\r\nThe board brings Persistence of Vision (POV) to life: fast LED sequences create floating text and patterns in the air. Everything is controlled by a RISC-V microcontroller whose firmware is written in Embedded Rust. Rust makes programming super intuitive. Anyone who has ever written Arduino code will love it. Text, symbols and animations can be easily customised without having to struggle through complicate

## 5. User Endpoints

In [14]:
# 5.1 Get User by ID
# Note: We need a valid user ID. This endpoint requires a UUID.
print("🔍 Testing: GET /users/{id}")
print("⚠️  Note: User endpoint requires a valid UUID.")
print("   In production, you would get user IDs from other endpoints or user authentication.")
print("   For testing, you would need to provide a real user UUID.")
print("   Example usage (uncomment and replace with real ID):")
print("   # sample_user_id = '0f7d0b92-1660-4734-bdc2-af95b62001b9'")
print("   # user_data = make_request(f'/users/{sample_user_id}')")
print("   # print_response(f'User Detail (ID: {sample_user_id})', user_data)")
print("   Skipping user detail test - requires a valid user UUID")

🔍 Testing: GET /users/{id}
⚠️  Note: User endpoint requires a valid UUID.
   In production, you would get user IDs from other endpoints or user authentication.
   For testing, you would need to provide a real user UUID.
   Example usage (uncomment and replace with real ID):
   # sample_user_id = '0f7d0b92-1660-4734-bdc2-af95b62001b9'
   # user_data = make_request(f'/users/{sample_user_id}')
   # print_response(f'User Detail (ID: {sample_user_id})', user_data)
   Skipping user detail test - requires a valid user UUID


## 6. Badge Endpoints

In [15]:
# 6.1 List Badges
print("🔍 Testing: GET /badges/")
badges_data = make_request("/badges/", params={"pageSize": 10})
test_results['badges_list'] = badges_data
print_response("Badges List", badges_data)

# Extract first badge ID for detail endpoint
first_badge_id = None
if badges_data and 'data' in badges_data and len(badges_data['data']) > 0:
    first_badge_id = badges_data['data'][0].get('id')
    print(f"✅ First Badge ID: {first_badge_id}")

🔍 Testing: GET /badges/
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 68 bytes

📋 Badges List
{
  "data": [],
  "pagination": {
    "page_size": 10,
    "page": 1,
    "total": 0
  }
}

📊 Summary: 0 items returned
   Page: 1/1
   Total: 0 items



In [16]:
# 6.2 Get Badge by ID
if first_badge_id:
    print(f"🔍 Testing: GET /badges/{first_badge_id}/")
    badge_detail = make_request(f"/badges/{first_badge_id}/")
    test_results['badge_detail'] = badge_detail
    print_response(f"Badge Detail (ID: {first_badge_id})", badge_detail)
else:
    print("⚠️  Skipping badge detail test - no badge ID available")

⚠️  Skipping badge detail test - no badge ID available


In [17]:
# 6.3 Test Badge Filtering Examples
print("🔍 Testing: Badge filtering examples")

# Filter by category
print(f"\n📌 Filtering badges by category")
badges_by_category = make_request("/badges/", params={
    "category": ["assembly"],
    "pageSize": 5
})
if badges_by_category and 'data' in badges_by_category:
    print(f"   Found {len(badges_by_category['data'])} badges in 'assembly' category")

# Filter by assembly
if first_assembly_id:
    print(f"\n📌 Filtering badges by assembly ID: {first_assembly_id}")
    badges_by_assembly = make_request("/badges/", params={
        "assemblyID": [first_assembly_id],
        "pageSize": 5
    })
    if badges_by_assembly and 'data' in badges_by_assembly:
        print(f"   Found {len(badges_by_assembly['data'])} badges for this assembly")

🔍 Testing: Badge filtering examples

📌 Filtering badges by category
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 67 bytes
   Found 0 badges in 'assembly' category

📌 Filtering badges by assembly ID: 000e8037-1af2-4951-8d4d-58737b6d8ad5
   Status Code: 200
   Content-Type: application/json; charset=utf-8
   Response Length: 67 bytes
   Found 0 badges for this assembly


## Test Summary

In [18]:
# Print summary of all tests
print("\n" + "="*60)
print("📊 TEST SUMMARY")
print("="*60)

endpoints_tested = {
    "Conference": ["/conference/", "/conference/tracks/", "/conference/tracks/{id}/"],
    "Assemblies": ["/assemblies/", "/assemblies/{id}/", "/assemblies/{id}/rooms/", "/assemblies/{id}/rooms/{room_id}/"],
    "Events": ["/events/", "/events/{id}/"],
    "Projects": ["/projects/", "/projects/{id}/"],
    "Users": ["/users/{id}"],
    "Badges": ["/badges/", "/badges/{id}/"]
}

for category, endpoints in endpoints_tested.items():
    print(f"\n{category}:")
    for endpoint in endpoints:
        # Check if we have test results for this endpoint
        status = "✅"
        if "{id}" in endpoint or "{room_id}" in endpoint:
            status = "⚠️  (requires ID)"
        print(f"  {status} {endpoint}")

print("\n" + "="*60)
total_endpoints = sum(len(eps) for eps in endpoints_tested.values())
print(f"Total endpoints tested: {total_endpoints}")
print("="*60)

# Show which tests returned data
print("\n📋 Tests with successful responses:")
successful_tests = 0
for test_name, result in test_results.items():
    if result is not None:
        print(f"  ✅ {test_name}")
        successful_tests += 1
    else:
        print(f"  ❌ {test_name}")

print(f"\n✅ Successful: {successful_tests}/{len(test_results)}")


📊 TEST SUMMARY

Conference:
  ✅ /conference/
  ✅ /conference/tracks/
  ⚠️  (requires ID) /conference/tracks/{id}/

Assemblies:
  ✅ /assemblies/
  ⚠️  (requires ID) /assemblies/{id}/
  ⚠️  (requires ID) /assemblies/{id}/rooms/
  ⚠️  (requires ID) /assemblies/{id}/rooms/{room_id}/

Events:
  ✅ /events/
  ⚠️  (requires ID) /events/{id}/

Projects:
  ✅ /projects/
  ⚠️  (requires ID) /projects/{id}/

Users:
  ⚠️  (requires ID) /users/{id}

Badges:
  ✅ /badges/
  ⚠️  (requires ID) /badges/{id}/

Total endpoints tested: 14

📋 Tests with successful responses:
  ❌ conference
  ✅ tracks_list
  ✅ track_detail
  ✅ assemblies_list
  ✅ assembly_detail
  ✅ assembly_rooms_list
  ✅ events_list
  ✅ event_detail
  ✅ projects_list
  ✅ project_detail
  ✅ badges_list

✅ Successful: 10/11
